In [1]:
import  os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")
os.environ['LANGCHAIN_API_KEY']=os.getenv('LANGCHAIN_API_KEY')
os.environ['LANGCHAIN_TRACING_V2']="true"
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")

In [4]:
## data  ingestion (scraping)

from langchain_community.document_loaders import WebBaseLoader

In [5]:
loader=WebBaseLoader("https://docs.langchain.com/langsmith/billing")

In [6]:
loader

In [8]:
docs=loader.load()
docs

[Document(metadata={'source': 'https://docs.langchain.com/langsmith/billing', 'title': 'Manage billing in your account - Docs by LangChain', 'language': 'en'}, page_content='Manage billing in your account - Docs by LangChainOur new LangChain Academy Course Deep Research with LangGraph is now live! Enroll for free.Docs by LangChain home pagePythonSearch...⌘KLangSmithPlatform for LLM observability and evaluationSetupOverviewCreate an account and API keySet up a workspaceManage organizations using the APIManage billingSet up resource tagsUser managementAdditional resourcesFAQsTroubleshootingCloud architecture and scalabilityRegions FAQAuthentication methodsData purging for complianceRelease versionsOur new LangChain Academy Course Deep Research with LangGraph is now live! Enroll for free.Docs by LangChain home pagePythonSearch...⌘KAsk AIGitHubForumForumSearch...NavigationSetupManage billing in your accountGet startedObservabilityEvaluationPrompt engineeringSelf-hostingAdministrationGet st

In [10]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
documents=text_splitter.split_documents(docs)

In [11]:
documents

[Document(metadata={'source': 'https://docs.langchain.com/langsmith/billing', 'title': 'Manage billing in your account - Docs by LangChain', 'language': 'en'}, page_content='Manage billing in your account - Docs by LangChainOur new LangChain Academy Course Deep Research with LangGraph is now live! Enroll for free.Docs by LangChain home pagePythonSearch...⌘KLangSmithPlatform for LLM observability and evaluationSetupOverviewCreate an account and API keySet up a workspaceManage organizations using the APIManage billingSet up resource tagsUser managementAdditional resourcesFAQsTroubleshootingCloud architecture and scalabilityRegions FAQAuthentication methodsData purging for complianceRelease versionsOur new LangChain Academy Course Deep Research with LangGraph is now live! Enroll for free.Docs by LangChain home pagePythonSearch...⌘KAsk AIGitHubForumForumSearch...NavigationSetupManage billing in your accountGet startedObservabilityEvaluationPrompt engineeringSelf-hostingAdministrationGet st

In [12]:
from langchain_openai import OpenAIEmbeddings
embeddings=OpenAIEmbeddings()

In [13]:
from langchain_community.vectorstores import FAISS

vectorstoredb=FAISS.from_documents(documents,embeddings)

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [14]:
query='''Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}
Output is truncated'''

In [ ]:
#quering from vector  store   db

result=vectorstoredb.similarity_search(query)
result[0].page_content

NameError: name 'vectorstoredb' is not defined

In [19]:
from langchain_openai import ChatOpenAI
llm=ChatOpenAI(model="gpt-4o")

In [20]:
## retrieval chain

from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

prompt=ChatPromptTemplate.from_template(
    '''
answer the following question based only on the provided context:
<context>
{context}
</context>
'''
)

In [22]:
document_chain=create_stuff_documents_chain(llm,prompt)
document_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nanswer the following question based only on the provided context:\n<context>\n{context}\n</context>\n'), additional_kwargs={})])
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x71645d403640>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x71645d403df0>, root_client=<openai.OpenAI object at 0x71645e89f880>, root_async_client=<openai.AsyncOpenAI object at 0x71645d4036d0>, model_name='gpt-4o', model_kwargs={}, openai_api_key=SecretStr('**********'))
| StrOutputParser(), kwargs={}, config={'run_name': 'stuf

In [23]:
from langchain_core.documents import Document

document_chain.invoke({
    "input":"Modify invoice email addresses, business information, and tax IDs for your organization.",
    "context":[Document(page_content='''Set up billing for your account: Complete the billing setup process for Developer and Plus plans, including special instructions for legacy accounts.
Update your information: Modify invoice email addresses, business information, and tax IDs for your organization.
Optimize your tracing spend: Learn how to reduce costs through data retention management and usage limits.''')
]})

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [25]:
retriever=vectorstoredb.as_retriever()

NameError: name 'vectorstoredb' is not defined

In [26]:
from langchain.chains import create_retrieval_chain
retrieval_chain=create_retrieval_chain(retriever,document_chain)

NameError: name 'retriever' is not defined

In [30]:
##  response from the llm


response=retrieval_chain.invoke({"input":"Modify invoice email addresses, business information, and tax IDs for your organization"})
response['answer']

NameError: name 'retrieval_chain' is not defined

In [31]:
response['context']

NameError: name 'response' is not defined